In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import torch
import pandas as pd
import numpy as np
import torch.nn as nn
from sklearn.model_selection import train_test_split 
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import MinMaxScaler

In [3]:
#Set the random seed

torch.manual_seed(2022)

In [4]:
cancer = pd.read_csv("/content/drive/MyDrive/Cases/Wisconsin/BreastCancer.csv")
dum_cancer = pd.get_dummies(cancer,drop_first=True)
dum_cancer.head()

,Code,Clump,UniCell_Size,Uni_CellShape,MargAdh,SEpith,BareN,BChromatin,NoemN,Mitoses,Class_Malignant
0,61634,5,4,3,1,2,2,2,3,1,0
1,63375,9,1,2,6,4,10,7,7,2,1
2,76389,10,4,7,2,2,8,6,1,1,1
3,95719,6,10,10,10,8,10,7,10,7,1
4,128059,1,1,1,1,2,5,5,1,1,0


In [5]:
X = dum_cancer.iloc[:,1:-1]

scaler = MinMaxScaler()

y = dum_cancer.iloc[:,-1].values
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, 
                                                    random_state=42,stratify=y)
X_scl_trn = scaler.fit_transform(X_train) 
X_scl_tst = scaler.transform(X_test) 


In [6]:
X_torch = torch.from_numpy(X_scl_trn)
y_torch = torch.from_numpy(y_train)
print(X_torch.size())
print(y_torch.size())

torch.Size([489, 9])
torch.Size([489])


In [7]:
type(X_torch)

torch.Tensor

In [9]:
# Create a model
model = nn.Sequential(nn.Linear(in_features=X_scl_trn.shape[1], out_features=5),
                      nn.ReLU(),
                      nn.Linear(5, 3),
                      nn.ReLU(),
                      nn.Linear(3,1),
                      nn.Sigmoid())

In [10]:
criterion = torch.nn.BCELoss()
# Construct the optimizer (Adam in this case)
optimizer = torch.optim.Adam(model.parameters(), lr = 0.001)
optimizer

Adam (
Parameter Group 0
    amsgrad: False
    betas: (0.9, 0.999)
    capturable: False
    differentiable: False
    eps: 1e-08
    foreach: None
    fused: False
    lr: 0.001
    maximize: False
    weight_decay: 0
)

In [11]:
y_pred = model(X_torch.float())
y_torch = y_torch.unsqueeze(1)
print(y_torch.shape)
print(y_pred.shape)

torch.Size([489, 1])
torch.Size([489, 1])


In [12]:
# Gradient Descent

for epoch in np.arange(0,1000):
   # Forward pass: Compute predicted y by passing x to the model
   y_pred_prob = model(X_torch.float())

   # Compute and print loss
   loss = criterion(y_pred_prob, y_torch.float())
   print('epoch: ', epoch+1,' loss: ', loss.item())

   # Zero gradients, perform a backward pass, and update the weights.
   optimizer.zero_grad()

   # perform a backward pass (backpropagation)
   loss.backward()

   # Update the parameters
   optimizer.step()
#print('epoch: ', epoch+1,' loss: ', loss.item())

893  loss:  0.09307719022035599
epoch:  894  loss:  0.09304314851760864
epoch:  895  loss:  0.09300896525382996
epoch:  896  loss:  0.09297468513250351
epoch:  897  loss:  0.0929403230547905
epoch:  898  loss:  0.09290594607591629
epoch:  899  loss:  0.09287156909704208
epoch:  900  loss:  0.09283715486526489
epoch:  901  loss:  0.09280011802911758
epoch:  902  loss:  0.09276249259710312
epoch:  903  loss:  0.09272482991218567
epoch:  904  loss:  0.0926871970295906
epoch:  905  loss:  0.09264948219060898
epoch:  906  loss:  0.09261174499988556
epoch:  907  loss:  0.09257441759109497
epoch:  908  loss:  0.09253815561532974
epoch:  909  loss:  0.09250213950872421
epoch:  910  loss:  0.09246636182069778
epoch:  911  loss:  0.09243080765008926
epoch:  912  loss:  0.09239546954631805
epoch:  913  loss:  0.09236035495996475
epoch:  914  loss:  0.09232544898986816
epoch:  915  loss:  0.09229070693254471
epoch:  916  loss:  0.09225622564554214
epoch:  917  loss:  0.09222307801246643
epoch:  91

In [13]:
X_torch_test = torch.from_numpy(X_scl_tst)
y_pred_prob = model(X_torch_test.float()) # Equivalent predict_proba / predict
y_pred_prob = y_pred_prob.detach().numpy()
y_pred_prob = y_pred_prob.reshape(y_test.shape[0],)
y_pred_prob[:5]

array([0.00911858, 0.00730865, 0.00632951, 0.97004086, 0.68660223],
      dtype=float32)

In [15]:
y_pred = np.where(y_pred_prob >= 0.5,1,0)

y_pred[:5]

array([0, 0, 0, 1, 1])

In [ ]:
print(accuracy_score(y_test,y_pred))